In [15]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import json
from data_loader import DataFetcher
from auxiliary import multask
from indicators import *
import multiprocessing
import re
import os
from Minute_factor_calculator import *
from indicator_analyzer import *
import multiprocessing as mp
from functools import partial


import matplotlib.pyplot as plt 
%matplotlib inline 
plt.style.use('seaborn-whitegrid')
plt.rcParams['font.family'] = 'SimHei'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (21, 10) 

plt.rcParams.update({'font.size':18})
plt.rc('xtick', labelsize=12) 
plt.rc('ytick', labelsize=16) 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


C:\Users\zhengh\AppData\Local\Temp\ipykernel_5852\571802857.py:19: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


## 加载数据

In [11]:
## 加载数据和初始化数据接口
tradingday = pd.read_csv("all_tradeday.csv",index_col=0,parse_dates=[0])
data_fetcher = DataFetcher()
freq_list = ['Open-1DOpen', 'Open-2DOpen', 'Open-3DOpen', 'Open-4DOpen', 'Open-5DOpen',
                  'Open-1DClose', 'Open-5DClose',
                  'MorningOpen-1DMorningOpen', 'MorningOpen-2DMorningOpen', 'MorningOpen-3DMorningOpen',
                  'MorningOpen-4DMorningOpen', 'MorningOpen-5DMorningOpen', ]
his_data = {}
for k in freq_list:
    his_data[k] = pd.read_csv("ret_cache/" + k + ".csv", index_col=0, parse_dates=[0])


## 编辑因子表达式

In [12]:
start_date = "2015-01-01"
end_date = "2023-01-01"
datelist = list(tradingday.loc[start_date:end_date].index)
factor_formula = {
    'rv_umd':  'up_down_diff(ret(get_day_period(close(df)),1),0,std)',
    'rv_umd_u':  'up_down_diff_u(ret(get_day_period(close(df)),1),0,std)',
    'rv_umd_d':  'up_down_diff_d(ret(get_day_period(close(df)),1),0,std)',
    'rv_umd_ud':  'up_down_diff_ud(ret(get_day_period(close(df)),1),0,std)',
}
factor_rolling_method = {
    'rv_umd': 'MA-10',
}

task_list = { key:[] for key in factor_formula.keys()}


In [13]:
# 生成
for date in datelist:
    # 加载当日量价数据,统一加载
    dominant_code = data_fetcher.get_dominant_code(date)
    data = data_fetcher.get_price_ohlc_data(date = date)
    for factor,inputstr in factor_formula.items():
        if factor in ['cangdan','cangdan_huanbi']:
            data_input = data_fetcher.get_cangdan_data(date)
        elif factor in ['member_oi']:
            data_input = data_fetcher.get_member_oi_data(date)
        elif factor in ['jicha_mom']:
            data_input = data_fetcher.get_yuecha_ratio(date,method='raw_mom')
        elif factor in ['kucun_tongbi','kucun_shuiwei','kucun_huanbi_week','kucun_huanbi_month','kucun_huanbi_diff_week','kucun_huanbi_diff_month']:
            data_input = data_fetcher.get_kucun_data(date,method=factor[6:])
        else:
            data_input = data
        temp = {
            'date': date,
            'df': data_input,
            'inputstr': inputstr,
            'codes': dominant_code,
        }
        task_list[factor].append(temp)



## 因子计算，输出分组信息

In [14]:
for key,tasks in task_list.items():
    print(key)
    res = multask(tasks=tasks, func=parse_func)
    res = pd.concat(res,axis=0)
    res = res.sort_index()
    res = processing_ind(res,factor_rolling_method[key])
    group_res = res.apply(lambda x:handle_one_bar(x,5),axis=1)
    file_dir = f'factor_res/{key}'
    if not os.path.exists(file_dir):
        os.makedirs(file_dir, exist_ok=True)

    res.to_csv(file_dir + f"/{key}_val.csv")
    group_res.to_csv(file_dir + f"/{key}_group.csv")

rv_umd


ValueError: No objects to concatenate

## 生成报告

In [ ]:
factors = list(factor_formula.keys())
generator_report_partial = partial(generator_report,data=his_data,start_date="2015-01-01",end_date='2023-01-01')
pool = mp.Pool(processes=8)
pool.map(generator_report_partial,factors)
pool.close()
pool.join()